In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

# Ignore Harmless Warnings
import warnings
warnings.filterwarnings("ignore")

In [11]:
# Data Augmentation for training dataset

train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(), # Default probability of 0.5
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], # Mean and Standard Deviation for each colour channels. Helps in faster convergence
                        [0.229, 0.224, 0.225])
])

In [12]:
# Data Augmentation for the testing dataset
# We dont need to perform random rotations or flips in the test dataset

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])
])

In [13]:
root = 'C:\\Users\\shant\\CNN_Architectures_ImageNet_Pytorch\\Dataset\\CATS_DOGS\\'

# Location of the training data
train_data = datasets.ImageFolder(os.path.join(root,'train'),transform = train_transform)

# Location of the testing data
test_data = datasets.ImageFolder(os.path.join(root,'test'),transform = test_transform)

# Set a seed
torch.manual_seed(42)

train_loader = DataLoader(train_data, batch_size = 10, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 10)

In [14]:
class AlexNet(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 96, kernel_size = 11, stride = 4, padding = 2)
        self.conv1_bn = nn.BatchNorm2d(96)
        self.conv2 = nn.Conv2d(in_channels = 96 , out_channels = 256 , kernel_size = 5 , padding = 2)
        self.conv2_bn = nn.BatchNorm2d(256)
        self.conv3 = nn.Conv2d(in_channels = 256, out_channels = 384, kernel_size = 3 , padding = 1)
        self.conv4 = nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = 3 , padding = 1)
        self.conv5 = nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = 3 , padding = 1)
        self.conv5_bn = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(6*6*256, 4096)
        self.fc2 = nn.Linear(4096,4096)
        self.dropout = nn.Dropout(p = 0.5)
        self.fc3 = nn.Linear(4096,2)
        
    def forward(self, X):
        
        X = F.relu(F.max_pool2d(self.conv1_bn(self.conv1(X)), 3, 2))
        X = F.relu(F.max_pool2d(self.conv2_bn(self.conv2(X)), 3, 2))
        X = F.relu(self.conv3(X))
        X = F.relu(self.conv4(X))
        X = F.relu(F.max_pool2d(self.conv5_bn(self.conv5(X)), 3, 2))
        X = X.view(-1, 6*6*256)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.dropout(X)
        X = self.fc3(X)
        
        return F.log_softmax(X,dim = 1)

In [15]:
# Set a Seed
torch.manual_seed(101)

# Create an Instance of the class
AlexNetModel = AlexNet()

# Declare the Criterion
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(AlexNetModel.parameters(), lr = 0.001)

In [16]:
AlexNetModel

AlexNet(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (conv1_bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2_bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=9216, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=4096, out_features=2, bias=True)
)

In [17]:
# Training the Model

# Training and Evaluation of the Model

import time
start_time = time.time()

epochs = 2

# Limits on the number of batches
max_trn_batch = 800 # Batch of 10 images--> 8000 Images
max_tst_batch = 300 # Batch of 10 images--> 3000 Images

train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    
    trn_corr = 0
    tst_corr = 0
    
    for b, (X_train,y_train) in enumerate(train_loader):
    
        b += 1
        
        # Predicted Output
        y_pred = AlexNetModel(X_train)
        
        # Predicted Loss
        loss = criterion(y_pred,y_train)
        
        # Tally number of correct predictions
        predicted = torch.max(y_pred.data,1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        # Update the Parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if b%200 == 0:
            print(f'Epoch {i} LOSS {loss.item()}')
    
    train_losses.append(loss)
    train_correct.append(trn_corr)
    
    #TEST SET
    
    with torch.no_grad():
        
        for b , (X_test,y_test) in enumerate(test_loader):
            
            y_val = AlexNetModel(X_test)
            
            predicted = torch.max(y_pred.data, 1)[1] # Give max value along each row and then present as a column vector
            batch_corr = (predicted == y_test).sum()
            tst_corr = batch_corr
            
    loss = criterion(y_val,y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)
    
total_time = time.time() - start_time
print(f'Total Time: {total_time/60} minutes')

Epoch 0 LOSS 0.7528567314147949
Epoch 0 LOSS 0.6952638626098633
Epoch 0 LOSS 0.7005386352539062
Epoch 0 LOSS 0.6746869087219238
Epoch 0 LOSS 0.6904305815696716
Epoch 0 LOSS 0.6937099099159241
Epoch 0 LOSS 0.6864084005355835
Epoch 0 LOSS 0.6937990188598633
Epoch 0 LOSS 0.6939499378204346


RuntimeError: The size of tensor a (3) must match the size of tensor b (10) at non-singleton dimension 0